<a href="https://colab.research.google.com/github/gvogiatzis/CS4740/blob/main/CS4740_Lab_Week_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
from textblob import Word
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [2]:
! wget https://github.com/suraj-deshmukh/BBC-Dataset-News-Classification/raw/master/dataset/dataset.csv -O dataset.csv
! python -m textblob.download_corpora

--2021-01-21 15:34:58--  https://github.com/suraj-deshmukh/BBC-Dataset-News-Classification/raw/master/dataset/dataset.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/suraj-deshmukh/BBC-Dataset-News-Classification/master/dataset/dataset.csv [following]
--2021-01-21 15:34:59--  https://raw.githubusercontent.com/suraj-deshmukh/BBC-Dataset-News-Classification/master/dataset/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5108610 (4.9M) [text/plain]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>]   4.87M  21.9MB/s    in 0.2s    

2021-01-21 15:35:00 (21.9 MB/s) - ‘

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"\'s", "", string)
    string = re.sub(r"\'ve", "", string)
    string = re.sub(r"n\'t", "", string)
    string = re.sub(r"\'re", "", string)
    string = re.sub(r"\'d", "", string)
    string = re.sub(r"\'ll", "", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", "", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"'", "", string)
    string = re.sub(r"`", "", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[0-9]\w+|[0-9]","", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

data = pd.read_csv('dataset.csv', encoding = "ISO-8859-1")
doctxt = data['news'].tolist()
doctype = data['type'].tolist()

docs= [[Word(w).lemmatize() for w in clean_str(s).split()] for s in doctxt]
index_to_term = sorted(list(set().union(*docs)))
vocab_size = len(index_to_term)
term_to_index = {t:i for i,t in enumerate(index_to_term)}

data = [[ term_to_index[w] for w in doc] for doc in docs]

In [18]:
vocab_size = len(index_to_term)
embed_size = 300
hidden_dim=128

embedding = nn.Embedding(vocab_size, embed_size)
lstm = nn.LSTM(input_size=embed_size,
                            hidden_size=hidden_dim,
                            num_layers=1,
                            batch_first=True)

In [8]:
len(data[0])

445

In [20]:
x = torch.tensor(data[0])
x = embedding(x.unsqueeze(0))
x.shape
y,h = lstm(x)
y.shape

torch.Size([1, 445, 128])

In [25]:
h[1].shape

torch.Size([1, 1, 128])

In [ ]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, embed_size=300, hidden_dim=128):
        super(TextClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size=embed_size,
                            hidden_size=hidden_dim,
                            num_layers=1,
                            batch_first=True)
        
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):

        x = self.embedding(x)
        self.lstm(x)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)

        return text_out